In [12]:
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, Column, Integer, String, Float, DateTime
Base = declarative_base()
import os

In [13]:
class Measurement(Base):
    __tablename__ = 'measurement'
    measurement_id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(DateTime)
    prcp = Column(Float)
    tobs = Column(Integer)

In [14]:
class Station(Base):
    __tablename__ = 'station'
    station_id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [19]:
# Read csv
measurements_df = pd.read_csv('Resources\\clean_hawaii_measurements.csv')
measurements_df.rename(columns={'Unnamed: 0':'measurement_id'},inplace=True)
measurements_df.set_index(keys='measurement_id',inplace=True)
for index, row in measurements_df.iterrows():
    measurements_df.set_value(index, 'date', dt.datetime.strptime(row['date'],'%Y-%m-%d'))

stations_df = pd.read_csv('Resources\\clean_hawaii_stations.csv')
stations_df.rename(columns={'Unnamed: 0':'station_id'},inplace=True)
stations_df.set_index(keys='station_id',inplace=True)

In [20]:
measurements_df.head()

,station,date,prcp,tobs
measurement_id,,,,
0,USC00519397,2010-01-01 00:00:00,0.08,65
1,USC00519397,2010-01-02 00:00:00,0.00,63
2,USC00519397,2010-01-03 00:00:00,0.00,74
3,USC00519397,2010-01-04 00:00:00,0.00,76
5,USC00519397,2010-01-07 00:00:00,0.06,70


In [21]:
# Create/Update db and tables
engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()
conn.execute('DROP TABLE measurement')
conn.execute('DROP TABLE station')
Base.metadata.create_all(engine)
session = Session(bind=engine)
session.commit()

measurements_df.to_sql(name='measurement_temp',con=conn,if_exists='replace')
stations_df.to_sql(name='station_temp',con=conn,if_exists='replace')
conn.execute('INSERT INTO measurement SELECT * FROM measurement_temp')
conn.execute('INSERT INTO station SELECT * FROM station_temp')
conn.execute('DROP TABLE measurement_temp')
conn.execute('DROP TABLE station_temp')
conn.close()
